In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import statistics as s
import re

In [2]:
from functions import log

In [3]:
numFiles = 2

fileList_list = []

for i in range (0, numFiles):
    fileList_list.append('files/file_list_'+str(i)+'.txt')

In [4]:
all_df = []
for file in fileList_list:
    df = pd.read_csv(file, delimiter= '\t')
    all_df.append(df)

df = pd.concat(all_df)
    

In [5]:
df = df[df["Article Citation"].str.startswith("PLoS")]
df = df[["Article File", "PMID"]]
df = df[df["PMID"] != 0]
df.head()

,Article File,PMID
0,PMC000xxxxxx/PMC176545.xml,12929205
1,PMC000xxxxxx/PMC176546.xml,12929206
4,PMC000xxxxxx/PMC193604.xml,12975658
5,PMC000xxxxxx/PMC193605.xml,12975657
9,PMC000xxxxxx/PMC212687.xml,14551903


In [6]:
# initialize empty lists
titles = []
abstracts = []
refs = []

In [7]:
def main(article_file):
    tree = ET.parse('./files/'+article_file) # parse file
    root = tree.getroot() # get root
    
    # get title
    title_element = root.findall('front')[0].findall('article-meta')[0].findall('title-group')[0].findall('article-title')[0]
    title = "".join(title_element.itertext())
    title = title.replace('\n','')
    
    # get abstract
    try:
        abstract_element = root.findall('front')[0].findall('article-meta')[0].findall('abstract')[0]
        abstract = "".join(abstract_element.itertext())
        abstract = abstract.replace('\n','')
    except: # no abstract, take body?
        abstract_element = root.findall('body')[0]
        abstract = "".join(abstract_element.itertext())
        abstract = abstract.replace('\n','')
    
    # get references
    pmid_list = []
    try:
        reference_element = root.findall('back')[0].findall('ref-list')[0].findall('ref')
        counter = 0
        for ref in reference_element:
            try:
                pmid = "".join(ref.findall('element-citation')[0].findall('article-title')[0].itertext())
                pmid_list.append(pmid)
#                 if len(pmid) < 10:
#                     pmid_list.append(pmid)
            except: # no pmid
                pass
    except:
        pmid_list = "NA"
        
    titles.append(title)
    abstracts.append(abstract)
    refs.append(pmid_list)
    
    log(f'Ran select_files.ipynb succesfully')
        

In [8]:
for i in df["Article File"]:
    main(i)

In [9]:
df["title"] = titles
# df = df[["PMID", "title"]]
df["abstract"] = abstracts
df["references"] = refs

In [10]:
df = df[df["abstract"] != 'xx']
df = df[df["references"] != 'NA']
df = df.dropna()

In [11]:
def parseGroupString(ref):    
    newRef = [ re.sub('\n', '', y, flags=re.IGNORECASE) for y in ref ] 
    return newRef
                     
df["references"] = [ parseGroupString(x) for x in df["references"] ]

In [12]:
lens = df['abstract'].str.split().apply(len)
print(max(lens))
print(min(lens))
print(s.mean(lens))
print(s.stdev(lens))

6911
1
247.94698674668666
355.03607915118585


In [13]:
df.to_csv(r'files/select_files.txt', sep='\t', index = False)

In [14]:
df

,Article File,PMID,title,abstract,references
0,PMC000xxxxxx/PMC176545.xml,12929205,The Transcriptome of the Intraerythrocytic Dev...,Plasmodium falciparum is the causative agent o...,[Plasmodium falciparum: Analysis of transcribe...
1,PMC000xxxxxx/PMC176546.xml,12929206,DNA Analysis Indicates That Asian Elephants Ar...,The origin of Borneo's elephants is controvers...,[WHICHRUN (version 3.2): A computer program fo...
4,PMC000xxxxxx/PMC193604.xml,12975658,Drosophila Free-Running Rhythms Require Interc...,Robust self-sustained oscillations are a ubiqu...,[A mutant Drosophila homolog of mammalian Cloc...
5,PMC000xxxxxx/PMC193605.xml,12975657,From Gene Trees to Organismal Phylogeny in Pro...,The rapid increase in published genomic sequen...,[Genome sequence of the endocellular obligate ...
9,PMC000xxxxxx/PMC212687.xml,14551903,A Functional Analysis of the Spacer of V(D)J R...,"During lymphocyte development, V(D)J recombina...",[Distinct roles of RAG1 and RAG2 in binding th...
...,...,...,...,...,...
27441,PMC001xxxxxx/PMC1995761.xml,17925860,Large-Scale Mitochondrial DNA Analysis of the ...,"BackgroundFrom the beginning of domestication,...","[Domesticated animals from early times., Goats..."
27442,PMC001xxxxxx/PMC1995762.xml,17925881,"Endothelial Progenitor Cells, Cardiovascular R...",BackgroundEPC number and functionality are ass...,[Isolation of putative progenitor endothelial ...
27443,PMC001xxxxxx/PMC1995763.xml,17925862,Ataxin-1 Fusion Partners Alter PolyQ Lethality...,Intranuclear inclusion bodies (IBs) are the hi...,[Protein aggregation and neurodegenerative dis...
27444,PMC001xxxxxx/PMC1995764.xml,17925864,The Chemokine Receptor CXCR4 Strongly Promotes...,"Neuroblastoma (NB) is a heterogeneous, and par...",[Neuroblastoma: biological insights into a cli...
